# RUL Prediction with LSTM
b. Xs' and W by flight class
c. Xs' and W and $\theta$

1. Load flight effects model and validate performance
2. set up hyperparameter tuning for RUL prediction network (LSTM)
3. Evaluate different lookback windows or implement adaptive window

In [ ]:
import os
import sys
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

base_dir = os.path.dirname(os.getcwd())
print(base_dir)
sys.path.insert(1, base_dir)
from package.api import DB as api
import package.utils as utils
import package.tuning as tuning
utils.check_gpu()

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers, optimizers, metrics
#import tensorflow_addons as tfa

import keras_tuner as kt

from kerastuner_tensorboard_logger import (
    TensorBoardLogger,
    setup_tb  # Optional
)

paths_df = pd.read_csv(base_dir + '/paths.csv')
paths_df['path'] = base_dir + '/' + paths_df['path']

Fc = 3
dataset = 'DS08'
log_location = base_dir + '/logs'
model_location = base_dir + '/models'
data_location = base_dir + '/data'
data_header = f'Fc-{Fc}_dataset-{dataset}'


%matplotlib inline
%load_ext autoreload
%autoreload 2


# THESE ARE YOUR CREDENTIALS IN PLAIN TEXT!
params = {'datasource.username': 'macslab', # the username of the logged in user
            'datasource.password': 'Ch0colate!', 
            'datasource.database': 'ncmapss_db', # <- NO CHANGE 
            'datasource.url': '10.2.219.98', # <- or your database installation location
            'datasource.port': '5432'} # <- most likely don't change
#print(params)
db, cur =  api.connect(params)
db.set_session(autocommit=True)
del(params)

units_df = api._get_units(db=db)

units = units_df[(units_df['Fc'] == Fc) & (units_df['dataset'].str.contains(dataset))]

tables = ['summary_tb', 'telemetry_tb']
downsample=10
df = api._get_data(db=db,
                   units=pd.unique(units.id),
                   tables=tables,
                   downsample=downsample).astype(np.float32)
utils.add_time_column(units=pd.unique(units.id), df=df)
utils.add_rul_column(units=pd.unique(units.id), df=df)

W_cols = ['Mach', 'alt', 'TRA', 'T2', 'time']
Xs_cols = ['Wf', 'Nf', 'Nc', 'T24', 'T30', 'T48', 'T50', 'P15', 'P2', 'P21', 'P24', 'Ps30', 'P40', 'P50']
aux_cols = ['cycle', 'hs', 'Fc', 'asset_id']

model = keras.models.load_model(paths_df[paths_df['name']=='flight_effects'].path.values[0])
yscaler = joblib.load(paths_df[paths_df['name']=='flight_effects_yscaler'].path.values[0])
xscaler = joblib.load(paths_df[paths_df['name']=='flight_effects_xscaler'].path.values[0])

trace = yscaler.transform(df[Xs_cols])
pred = model.predict(xscaler.transform(df[W_cols]))
res = trace - pred
dfx = pd.DataFrame(data=res, columns=Xs_cols)
df_x = pd.DataFrame(data=xscaler.transform(df[W_cols]), columns=W_cols)
dfx = pd.concat([dfx, df_x, df[aux_cols]], axis=1)
dfx['rul'] = df['rul'].values
dfx.time = dfx.time + (dfx.cycle -1)
dfx0 = dfx[dfx.hs == 0]

train_df, train_y, val_df, val_y, test_df, test_y = utils.train_test_split(df=dfx0, units=list(pd.unique(units.id)), y_labels=['rul'], t_labels=aux_cols + Xs_cols + W_cols, train_pct = 1.0, val_pct=0.0, test_pct=0.0, verbose=True)
train_y = pd.DataFrame(data=train_y, columns=['rul'])
train_pop = pd.concat([train_df.pop(x) for x in ['asset_id', 'cycle', 'hs', 'Fc']], axis=1)

lookback = 500
horizon = 1
n_out = 1
n_features = len(important_features)

X_train, y_train = utils.temporalize_data(train_df[important_features].values, train_y.values, lookback, horizon, n_features, n_out)
train_pop_temp, y_pop = utils.temporalize_data(train_pop.values, train_y, lookback, horizon, len(train_pop.columns), n_out)

X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)



## 1. Make Xs_prime and y 

## 2. split sets and view feature distributions

In [19]:
train_df, train_y, val_df, val_y, test_df, test_y = utils.train_test_split(df=dfx0, units=list(pd.unique(units.id)), y_labels=['rul'], t_labels=aux_cols + Xs_cols + W_cols, train_pct = 1.0, val_pct=0.0, test_pct=0.0, verbose=True)

10
10
0
0
train, val, test set counts: 10, 0, 0
10 10
train units: [83, 85, 81, 71, 78, 67, 77, 82, 84, 79]
val units: []
test units: []


In [ ]:
train_y = pd.DataFrame(data=train_y, columns=['rul'])
val_y = pd.DataFrame(data=val_y, columns=['rul'])
test_y = pd.DataFrame(data=test_y, columns=['rul'])

In [ ]:
train_pop = pd.concat([train_df.pop(x) for x in ['asset_id', 'cycle', 'hs', 'Fc']], axis=1)
val_pop = pd.concat([val_df.pop(x) for x in ['asset_id', 'cycle', 'hs', 'Fc']], axis=1)
test_pop = pd.concat([test_df.pop(x) for x in ['asset_id', 'cycle', 'hs', 'Fc']], axis=1)

In [ ]:

train_df.to_csv(f'{data_location}/train_df_{data_header}.csv')
train_pop.to_csv(f'{data_location}/train_pop_{data_header}.csv')
train_y.to_csv(f'{data_location}/train_y_{data_header}.csv')

val_df.to_csv(f'{data_location}/val_df_{data_header}.csv')
val_pop.to_csv(f'{data_location}/val_pop_{data_header}.csv')
val_y.to_csv(f'{data_location}/val_y_{data_header}.csv')

test_df.to_csv(f'{data_location}/test_df_{data_header}.csv')
test_pop.to_csv(f'{data_location}/test_pop_{data_header}.csv')
test_y.to_csv(f'{data_location}/test_y_{data_header}.csv')

In [ ]:
#utils.plot_feature_distributions(df=train_df, feature_range=(0,1), figsize=(12,4))
#utils.plot_feature_distributions(df=val_df, feature_range=(0,1), figsize=(12,4))
#utils.plot_feature_distributions(df=test_df, feature_range=(0,1), figsize=(12,4))

### ~plot feature importances~

In [ ]:
# from xgboost import XGBRegressor
# importance_model = XGBRegressor()
# importance_model.fit(train_df.values, train_y)

# scoring = ['explained_variance',
#            'max_error']#,
#            #'neg_mean_absolute_error',
#            #'neg_root_mean_squared_error',
#            #'r2']

# utils.plot_feature_importances(model=importance_model,
#                                features=train_df.values,
#                                feature_labels=Xs_cols + W_cols,
#                                target=train_y,
#                                target_label='rul',
#                                scoring=scoring)

# utils.plot_feature_importance(features=train_df.values,
#                               feature_labels=Xs_cols + W_cols,
#                               target=train_y,
#                               target_label='rul',
#                               figsize=(9,4))

In [ ]:
# #important_features = list(train_df.columns) #['Wf','Nc', 'T24', 'T48', 'T50', 'P2', 'Ps30', 'P40']
# important_features = ['Wf','Nf', 'T30', 'T48', 'T50', 'P15', 'P24', 'P40', 'P50','TRA', 'T2', 'time']
# important_features
len(train_df.columns)

In [ ]:
X_train.shape

## 2. temporalize the data

In [ ]:
#important_features = list(train_df.columns)

lookback = 500
horizon = 1
n_out = 1
n_features = len(important_features)

X_train, y_train = utils.temporalize_data(train_df[important_features].values, train_y.values, lookback, horizon, n_features, n_out)
train_pop_temp, y_pop = utils.temporalize_data(train_pop.values, train_y, lookback, horizon, len(train_pop.columns), n_out)

X_val, y_val = utils.temporalize_data(val_df.values, val_y, lookback, horizon, n_features, n_out)
val_pop_temp, y_pop = utils.temporalize_data(val_pop.values, val_y, lookback, horizon, n_features, n_out)

X_test, y_test = utils.temporalize_data(test_df.values, test_y, lookback, horizon, n_features, n_out)
test_pop_temp, y_pop = utils.temporalize_data(test_pop.values, test_y, lookback, horizon, n_features, n_out)

In [ ]:
np.save(f'{data_location}/{data_header}_X.npy', X_train)
np.save(f'{data_location}/{data_header}_y.npy', y_train)

In [ ]:
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)

In [ ]:
X_train = np.load(f'{data_location}/{data_header}_X_train.npy').astype('float32')
y_train = np.load(f'{data_location}/{data_header}_y_train.npy').astype('float32')

X_test = np.load(f'{data_location}/{data_header}_X_test.npy').astype('float32')
y_test = np.load(f'{data_location}/{data_header}_y_test.npy').astype('float32')

X_val = np.load(f'{data_location}/{data_header}_X_val.npy').astype('float32')
y_val = np.load(f'{data_location}/{data_header}_y_val.npy').astype('float32')

print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

In [ ]:
X = np.vstack([X_train, X_test, X_val])
y = np.vstack([y_train, y_test, y_val])

## Parameter search

In [ ]:
X.shape[2]

In [ ]:

lookback = 100
horizon = 1
n_out = 1
n_features = X.shape[2]

footer = 'retry'
input_shape = (lookback, n_features)
my_tuning = tuning.Tuning(input_shape, n_out)
bayesian_tuning = my_tuning.bayesian_search(objective='val_root_mean_squared_error',
                                            mode='min',
                                            max_trials=96,
                                            alpha=.0005,
                                            beta=10,
                                            epochs=5,
                                            executions_per_trial=1,
                                            hypermodel=my_tuning.create_bilstm_hypermodel,
                                            directory=f'{log_location}/{data_header}_{footer}',
                                            project_name='bilstm',
                                            logger=TensorBoardLogger(
                                                metrics=['val_root_mean_squared_error'],
                                                         logdir=f'{log_location}/{data_header}_{footer}/hparams'
                                            ),
                                            X=X_train,
                                            y=y_train)

In [ ]:
bayesian_tuning_params = bayesian_tuning.get_best_hyperparameters(num_trials=1)[0]
bayesian_tuning_model = bayesian_tuning.get_best_models()[0]
print(bayesian_tuning_params.values)
bayesian_tuning_model.summary()

In [ ]:
bayesian_tuning_model.save(f'{model_location}/{data_header}_best.h5')
bayesian_tuning_model.save(f'{model_location}/{data_header}_best')

# scratch / misc below

In [ ]:
# Xs_prime = []
# idx = np.random.randint(len(Xs_cols))
# polys = []

# df[W_cols] = xscaler.transform(df[W_cols])
# pred = model.predict(df[W_cols])

# for asset_id in pd.unique(df.asset_id):
#     X_full = df[df.asset_id == asset_id][W_cols]
#     pred = model.predict(X_full)
#     # pred = model.predict(xscaler.transform(X_full))
#     Xs_prime.append(pred)
    
#    y_full = df[df.asset_id == asset_id][Xs_cols]
#    trace = yscaler.transform(y_full)
#     res = trace - pred

#     x = np.arange(0, len(res[:,0]))
#     x = x[0:len(x):10]
#     for i in range(0, len(Xs_cols)):
#         y = res[:,i]
#         y = y[0:len(y):10]
#         reg = np.poly1d(np.polyfit(x, y, 6))
#         polys.append(reg)

#     fig = plt.figure(figsize=(12,12))
#     filtered = np.zeros(res.shape)
#     x = np.arange(0, len(res[:,0]))
#     for j in range(len(Xs_cols)):
#         y = res[:,j]
#         p = polys[j]

#         diff = np.zeros((len(y),))
#         for i in range(len(y)):
#             predp = p(y[i])
#             diff[i] = y[i] - predp

#         mean = np.mean(diff)
#         std = np.std(diff)

#         filt = np.zeros((len(y),))
#         for i in range(len(res)):
#             if np.abs(np.abs(y[i]) - np.abs(p(x[i]))) > np.abs(mean)+3*std:
#                 filt[i] = p(x[i])
#             else:
#                 filt[i] = y[i]

#         filtered[:,j] = filt
#         utils.plot_trace_samples(fig, y, filt, p, j)
#     plt.show()
#     Xs_prime.append(pred)

In [ ]:
# import Queue
# from threading import Thread

# units = api._get_units(db=db)
# tables = ['summary_tb', 'telemetry_tb']

# def func(units, tables, downsample):
#     return str(units) + str(tables) + str(downsample)
# def delegate():
#     return func('units', ' tables', 20)
# units = api._get_units(db=db)
# data_frames = []
# for i in range(int(len(units)/10)):
    
    
    # df = api._get_data(db=db,
    #                    units=units,
    #                    tables=tables,
    #                    downsample=20).astype(np.float32)
    
    
# train_df = pd.read_csv('train_df.csv', index_col=0)
# train_pop = pd.read_csv('train_pop.csv', index_col=0)
# train_y = pd.read_csv('train_y.csv', index_col=0)

In [ ]:
from keras.utils.vis_utils import plot_model
input_shape = (lookback, n_features)
my_tuning = tuning.Tuning(input_shape, n_out)

params = tuning.MyParameters(layers=3, units=64, dropout_rate=0, learning_rate=.001, recurrent_dropout=0,l2=-1)


model = my_tuning.build_lstm_model(params)
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True)
model.summary()